In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


merged = pd.read_csv(r"soil_moisture_merged.csv")

print(f"Shape of merged dataset: {merged.shape}")
merged.head()

Shape of merged dataset: (2643, 18)


,AEZ_NAME,week_start,sm_surface,VV,VH,NDVI,LST,Rainfall_7d,Rainfall_7d_lag,ET,LAI,NDWI,NDMI,Temp_7d,Temp_7d_lagged,landcover,elevation,slope
0,Active Tista Floodplain,2022-01-01,0.159142,-5.761691,-13.478774,0.5488,22.31,3.705716,2.430984,8.2,0.6,0.041381,0.261766,17.507983,17.870370,40,33,1.386612
1,Tista Meander Floodplain,2022-01-01,0.152997,-7.386647,-16.112829,0.4492,23.41,4.648317,0.000000,7.5,0.6,-0.025512,0.127510,17.625635,17.970898,40,30,2.963912
2,Karatoya-Bangali Floodplain,2022-01-01,0.101049,-9.252055,-16.693300,0.4091,23.61,0.000000,0.000000,10.5,0.6,0.064735,0.241161,17.894887,18.837795,10,17,0.927410
3,Lower Atrai Basin,2022-01-01,0.107364,-9.480288,-18.177580,0.2117,23.75,6.558670,0.000000,10.5,0.2,0.022280,0.228830,17.792813,18.636809,40,13,1.378362
4,Active Brahmaputra-Jamuna Floodplain,2022-01-01,0.100783,-8.074512,-16.009851,0.4884,23.81,4.167820,2.591343,8.0,0.6,0.108474,0.320618,18.043685,18.809614,40,18,2.044861


In [3]:
# Missing value calculation
merged_rf = merged.copy()

# NDVI median imputation
ndvi_median = merged_rf['NDVI'].median()
merged_rf['NDVI'] = merged_rf['NDVI'].fillna(ndvi_median)

print(f"NDVI missing filled with median: {ndvi_median:.4f}")

NDVI missing filled with median: 0.5755


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Define features and target
features_rf = merged_rf.columns.difference(['sm_surface', 'AEZ_NAME', 'week_start'])
X_rf = merged_rf[features_rf]
y_rf = merged_rf['sm_surface']

# Create a pipeline (scaling is included for completeness, though RF doesn't require it)
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42))
])

# Define 5-fold CV
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for each metric
# Note: cross_val_score returns negative values for MSE by convention
cv_rmse_scores = np.sqrt(-cross_val_score(pipeline_rf, X_rf, y_rf, 
                                         cv=cv, scoring='neg_mean_squared_error'))
cv_mae_scores = -cross_val_score(pipeline_rf, X_rf, y_rf, 
                                 cv=cv, scoring='neg_mean_absolute_error')
cv_r2_scores = cross_val_score(pipeline_rf, X_rf, y_rf, 
                               cv=cv, scoring='r2')

# Compute mean and standard deviation
mean_rmse = cv_rmse_scores.mean()
std_rmse = cv_rmse_scores.std()

mean_mae = cv_mae_scores.mean()
std_mae = cv_mae_scores.std()

mean_r2 = cv_r2_scores.mean()
std_r2 = cv_r2_scores.std()

# Print results in desired format
print(f"Random Forest RMSE: {mean_rmse:.3f}±{std_rmse:.3f}")
print(f"Random Forest MAE: {mean_mae:.3f}±{std_mae:.3f}")
print(f"Random Forest R²: {mean_r2:.2f}±{std_r2:.3f}")

Random Forest RMSE: 0.043±0.002
Random Forest MAE: 0.032±0.001
Random Forest R²: 0.83±0.005


In [5]:
features_rf_5 = [
    'ET',
    'elevation',
    'Rainfall_7d_lag',
    'slope',
    'Temp_7d_lagged'
]

X_rf_5 = merged_rf[features_rf_5]
y_rf_5 = merged_rf['sm_surface']

# Create pipeline (includes scaling for consistency, though RF doesn't require it)
pipeline_rf_5 = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42))
])

# Define 5-fold CV
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validated metrics
cv_rmse_scores = np.sqrt(-cross_val_score(pipeline_rf_5, X_rf_5, y_rf_5, 
                                         cv=cv, scoring='neg_mean_squared_error'))
cv_mae_scores = -cross_val_score(pipeline_rf_5, X_rf_5, y_rf_5, 
                                 cv=cv, scoring='neg_mean_absolute_error')
cv_r2_scores = cross_val_score(pipeline_rf_5, X_rf_5, y_rf_5, 
                               cv=cv, scoring='r2')

# Compute mean and std
mean_rmse = cv_rmse_scores.mean()
std_rmse = cv_rmse_scores.std()

mean_mae = cv_mae_scores.mean()
std_mae = cv_mae_scores.std()

mean_r2 = cv_r2_scores.mean()
std_r2 = cv_r2_scores.std()

# Print results in desired format
print("== RF Model 2 (5 features) ==")
print(f"Random Forest RMSE: {mean_rmse:.3f}±{std_rmse:.3f}")
print(f"Random Forest MAE : {mean_mae:.3f}±{std_mae:.3f}")
print(f"Random Forest R²  : {mean_r2:.2f}±{std_r2:.3f}")

== RF Model 2 (5 features) ==
Random Forest RMSE: 0.057±0.002
Random Forest MAE : 0.043±0.001
Random Forest R²  : 0.71±0.018


In [9]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb

# ==== Features/target (as in your code) ====
features = merged_clean.columns.difference(['sm_surface', 'AEZ_NAME', 'week_start'])
X = merged_clean[features].values
y = merged_clean['sm_surface'].values

# ==== 5-fold CV setup ====
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

rmse_folds, mae_folds, r2_folds = [], [], []

fold_id = 1
for train_idx, val_idx in kf.split(X):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    # LightGBM model (sklearn API)
    model = lgb.LGBMRegressor(
        objective='regression',
        n_estimators=200,
        random_state=42
    )

    # Early stopping on the validation fold
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        early_stopping_rounds=20,
        verbose=False
    )

    # Predict using the best iteration
    y_pred = model.predict(X_val, num_iteration=model.best_iteration_)

    # Metrics for this fold
    rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
    mae  = float(mean_absolute_error(y_val, y_pred))
    r2   = float(r2_score(y_val, y_pred))

    rmse_folds.append(rmse)
    mae_folds.append(mae)
    r2_folds.append(r2)

    print(f"Fold {fold_id}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")
    fold_id += 1

# ==== Aggregate (mean ± SD) ====
def mean_sd(arr):
    arr = np.asarray(arr, dtype=float)
    mean = np.mean(arr)
    sd   = np.std(arr, ddof=1)  # sample SD
    return mean, sd

rmse_mean, rmse_sd = mean_sd(rmse_folds)
mae_mean,  mae_sd  = mean_sd(mae_folds)
r2_mean,   r2_sd   = mean_sd(r2_folds)

print("\n== 5-fold CV (LightGBM) ==")
print(f"RMSE: {rmse_mean:.4f} ± {rmse_sd:.4f}")
print(f"MAE : {mae_mean:.4f} ± {mae_sd:.4f}")
print(f"R²  : {r2_mean:.4f} ± {r2_sd:.4f}")

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument

Fold 1: RMSE=0.0421, MAE=0.0309, R²=0.8453
Fold 2: RMSE=0.0419, MAE=0.0311, R²=0.8412


C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument

Fold 3: RMSE=0.0390, MAE=0.0294, R²=0.8489
Fold 4: RMSE=0.0398, MAE=0.0295, R²=0.8560
Fold 5: RMSE=0.0409, MAE=0.0307, R²=0.8667

== 5-fold CV (LightGBM) ==
RMSE: 0.0408 ± 0.0013
MAE : 0.0303 ± 0.0008
R²  : 0.8516 ± 0.0101


C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [10]:
from sklearn.svm import SVR

# Data (same features/target as your script) ====
features = merged_clean.columns.difference(['sm_surface', 'AEZ_NAME', 'week_start'])
X = merged_clean[features].values
y = merged_clean['sm_surface'].values

# SVR with scaling inside the CV loop via Pipeline
best_C = 1
best_epsilon = 0.01
best_gamma = 0.1

svr_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf', C=best_C, epsilon=best_epsilon, gamma=best_gamma))
])

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

rmse_folds, mae_folds, r2_folds = [], [], []

fold = 1
for train_idx, val_idx in kf.split(X):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    svr_pipeline.fit(X_tr, y_tr)
    y_pred = svr_pipeline.predict(X_val)

    rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
    mae  = float(mean_absolute_error(y_val, y_pred))
    r2   = float(r2_score(y_val, y_pred))

    rmse_folds.append(rmse)
    mae_folds.append(mae)
    r2_folds.append(r2)

    print(f"Fold {fold}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")
    fold += 1

def mean_sd(arr):
    arr = np.asarray(arr, dtype=float)
    return float(np.mean(arr)), float(np.std(arr, ddof=1))  # sample SD

rmse_mean, rmse_sd = mean_sd(rmse_folds)
mae_mean,  mae_sd  = mean_sd(mae_folds)
r2_mean,   r2_sd   = mean_sd(r2_folds)

print("\n== 5-fold CV (SVR) ==")
print(f"RMSE: {rmse_mean:.4f} ± {rmse_sd:.4f}")
print(f"MAE : {mae_mean:.4f} ± {mae_sd:.4f}")
print(f"R²  : {r2_mean:.4f} ± {r2_sd:.4f}")

Fold 1: RMSE=0.0519, MAE=0.0367, R²=0.7652
Fold 2: RMSE=0.0483, MAE=0.0355, R²=0.7895
Fold 3: RMSE=0.0474, MAE=0.0349, R²=0.7769
Fold 4: RMSE=0.0466, MAE=0.0354, R²=0.8028
Fold 5: RMSE=0.0528, MAE=0.0377, R²=0.7776

== 5-fold CV (SVR) ==
RMSE: 0.0494 ± 0.0028
MAE : 0.0361 ± 0.0011
R²  : 0.7824 ± 0.0143


In [11]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

features_ann = [
    'VV', 'VH', 'NDVI', 'LST', 'Rainfall_7d', 'Rainfall_7d_lag',
    'ET', 'LAI', 'NDWI', 'NDMI', 'Temp_7d', 'Temp_7d_lagged',
    'landcover', 'elevation', 'slope'
]
X = merged_clean[features_ann].values
y = merged_clean['sm_surface'].values

def build_ann(input_dim: int):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='linear')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    return model

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

rmse_folds, mae_folds, r2_folds = [], [], []

fold = 1
for train_idx, val_idx in kf.split(X):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    scaler = StandardScaler()
    X_tr_sc = scaler.fit_transform(X_tr)
    X_val_sc = scaler.transform(X_val)

    model = build_ann(X_tr_sc.shape[1])
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )

    model.fit(
        X_tr_sc, y_tr,
        validation_split=0.1,
        epochs=200,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

    y_pred = model.predict(X_val_sc, verbose=0).flatten()

    rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
    mae = float(mean_absolute_error(y_val, y_pred))
    r2 = float(r2_score(y_val, y_pred))

    rmse_folds.append(rmse)
    mae_folds.append(mae)
    r2_folds.append(r2)

    print(f"Fold {fold}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")
    fold += 1

def mean_sd(arr):
    arr = np.asarray(arr, dtype=float)
    return float(np.mean(arr)), float(np.std(arr, ddof=1))

rmse_mean, rmse_sd = mean_sd(rmse_folds)
mae_mean, mae_sd = mean_sd(mae_folds)
r2_mean, r2_sd = mean_sd(r2_folds)

print("\n== 5-fold CV (ANN) ==")
print(f"RMSE: {rmse_mean:.4f} ± {rmse_sd:.4f}")
print(f"MAE : {mae_mean:.4f} ± {mae_sd:.4f}")
print(f"R²  : {r2_mean:.4f} ± {r2_sd:.4f}")

Fold 1: RMSE=0.0558, MAE=0.0406, R²=0.7288
Fold 2: RMSE=0.0492, MAE=0.0382, R²=0.7816
Fold 3: RMSE=0.0513, MAE=0.0395, R²=0.7396
Fold 4: RMSE=0.0540, MAE=0.0416, R²=0.7352
Fold 5: RMSE=0.0616, MAE=0.0457, R²=0.6971

== 5-fold CV (ANN) ==
RMSE: 0.0544 ± 0.0048
MAE : 0.0411 ± 0.0028
R²  : 0.7365 ± 0.0303


In [12]:
from sklearn.linear_model import LinearRegression

print("== Linear Regression (5-Fold CV) ==")

# Feature list
features_lr = [
    "ET", 
    "Rainfall_7d_lag", 
    "NDWI", 
    "Rainfall_7d", 
    "NDMI"
]

# Drop rows with missing target
merged_clean = merged.dropna(subset=['sm_surface'])

# Define X and y
X_lr = merged_clean[features_lr]
y_lr = merged_clean['sm_surface']

# Create pipeline: scaling + linear regression
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LinearRegression())
])

# Define 5-fold CV
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validated metrics
cv_rmse_scores = np.sqrt(-cross_val_score(pipeline_lr, X_lr, y_lr, 
                                         cv=cv, scoring='neg_mean_squared_error'))
cv_mae_scores = -cross_val_score(pipeline_lr, X_lr, y_lr, 
                                 cv=cv, scoring='neg_mean_absolute_error')
cv_r2_scores = cross_val_score(pipeline_lr, X_lr, y_lr, 
                               cv=cv, scoring='r2')

# Compute mean and standard deviation
mean_rmse = cv_rmse_scores.mean()
std_rmse = cv_rmse_scores.std()

mean_mae = cv_mae_scores.mean()
std_mae = cv_mae_scores.std()

mean_r2 = cv_r2_scores.mean()
std_r2 = cv_r2_scores.std()

# Print results in desired format
print(f"Linear Regression RMSE: {mean_rmse:.3f}±{std_rmse:.3f}")
print(f"Linear Regression MAE : {mean_mae:.3f}±{std_mae:.3f}")
print(f"Linear Regression R²  : {mean_r2:.2f}±{std_r2:.3f}")

== Linear Regression (5-Fold CV) ==
Linear Regression RMSE: 0.084±0.004
Linear Regression MAE : 0.064±0.002
Linear Regression R²  : 0.37±0.011
